# Amazon Kinesis Data Firehose

Amazon Kinesis Data Firehose is a fully managed service for delivering real-time streaming data to destinations such as Amazon S3, Amazon Redshift, Amazon Elasticsearch Service (Amazon ES), Splunk, and any custom HTTP endpoint. 

<img src="img/kinesis_firehose_transform.png" width="90%" align="left">

In [ ]:
import boto3
import sagemaker
import pandas as pd
import json

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)
firehose = boto3.Session().client(service_name='firehose', region_name=region)

In [ ]:
%store -r firehose_name

In [ ]:
try:
    firehose_name
except NameError:
    print('+++++++++++++++++++++++++++++++')
    print('[ERROR] Please run all previous notebooks in this section before you continue.')
    print('+++++++++++++++++++++++++++++++')

In [ ]:
print(firehose_name)

## Check IAM Roles Are In Place

In [ ]:
%store -r iam_kinesis_role_name

In [ ]:
try:
    iam_kinesis_role_name
except NameError:
    print('+++++++++++++++++++++++++++++++')
    print('[ERROR] Please run all previous notebooks in this section before you continue.')
    print('+++++++++++++++++++++++++++++++')

In [ ]:
print(iam_kinesis_role_name)

In [ ]:
%store -r iam_role_kinesis_arn

In [ ]:
try:
    iam_role_kinesis_arn
except NameError:
    print('+++++++++++++++++++++++++++++++')
    print('[ERROR] Please run all previous notebooks in this section before you continue.')
    print('+++++++++++++++++++++++++++++++')

In [ ]:
print(iam_role_kinesis_arn)

In [ ]:
%store -r iam_kinesis_role_passed

In [ ]:
try:
    iam_kinesis_role_passed
except NameError:
    print('+++++++++++++++++++++++++++++++')
    print('[ERROR] Please run all previous notebooks in this section before you continue.')
    print('+++++++++++++++++++++++++++++++')

In [ ]:
print(iam_kinesis_role_passed)

In [ ]:
if not iam_kinesis_role_passed:
    print('+++++++++++++++++++++++++++++++')
    print('[ERROR] Please run all previous notebooks in this section before you continue.')
    print('+++++++++++++++++++++++++++++++')
else:
    print('[OK]')

## Retrieve Lambda ARN

In [ ]:
%store -r lambda_fn_arn_invoke_ep

In [ ]:
try:
    lambda_fn_arn_invoke_ep
except NameError:
    print('+++++++++++++++++++++++++++++++')
    print('[ERROR] Please run all previous notebooks in this section before you continue.')
    print('+++++++++++++++++++++++++++++++')

In [ ]:
print(lambda_fn_arn_invoke_ep)

# Create a Kinesis Data Firehose Delivery Stream

In [ ]:
from botocore.exceptions import ClientError

try: 
    response = firehose.create_delivery_stream(
        DeliveryStreamName=firehose_name,
        DeliveryStreamType='DirectPut',
        ExtendedS3DestinationConfiguration={
           'RoleARN': iam_role_kinesis_arn,
           'BucketARN': 'arn:aws:s3:::{}'.format(bucket),
           'Prefix': 'kinesis-data-firehose/',  
           'ErrorOutputPrefix': 'kinesis-data-firehose-error/',
           'CompressionFormat': 'UNCOMPRESSED',
           'CloudWatchLoggingOptions': {
               'Enabled': True,
               'LogGroupName': '/aws/kinesisfirehose/dsoaws-kinesis-data-firehose',
               'LogStreamName': 'S3Delivery'
           },
           'ProcessingConfiguration': {
               'Enabled': True,
               'Processors': [{
                   'Type': 'Lambda',
                   'Parameters': [{
                       'ParameterName': 'LambdaArn',
                       'ParameterValue': '{}:$LATEST'.format(lambda_fn_arn_invoke_ep)
                   }]
               }]
           },
           'S3BackupMode': 'Enabled',
           'S3BackupConfiguration': {
               'RoleARN': iam_role_kinesis_arn,
               'BucketARN': 'arn:aws:s3:::{}'.format(bucket),
               'Prefix': 'kinesis-data-firehose-source-record/',  
               'ErrorOutputPrefix': '!{firehose:error-output-type}/',
               'CompressionFormat': 'UNCOMPRESSED'
           },
           'CloudWatchLoggingOptions': {
                'Enabled': False,
            }
       }
    )
    print('Delivery stream {} successfully created.'.format(firehose_name))
    print(json.dumps(response, indent=4, sort_keys=True, default=str))
except ClientError as e:
    if e.response['Error']['Code'] == 'ResourceInUseException':
        print('Delivery stream {} already exists.'.format(firehose_name))
    else:
        print('Unexpected error: %s' % e)

In [ ]:
import time

status = ''
while status != 'ACTIVE':    
    r = firehose.describe_delivery_stream(DeliveryStreamName=firehose_name)
    description = r.get('DeliveryStreamDescription')
    status = description.get('DeliveryStreamStatus')
    time.sleep(5)
    
print('Delivery Stream {} is active'.format(firehose_name))

## _This may take 1-2 minutes.  Please be patient._

In [ ]:
r = firehose.describe_delivery_stream(DeliveryStreamName=firehose_name)

status = description.get('DeliveryStreamStatus')
print(status)

print()

description = r.get('DeliveryStreamDescription')
print(json.dumps(description, indent=4, sort_keys=True, default=str))

In [ ]:
firehose_arn = r['DeliveryStreamDescription']['DeliveryStreamARN']
print(firehose_arn)

In [ ]:
%store firehose_arn

# Review Kinesis Data Firehose Delivery Stream

In [ ]:
from IPython.core.display import display, HTML
    
display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/firehose/home?region={}#/details/{}/details"> Firehose</a></b>'.format(region, firehose_name)))


# Store Variables for the Next Notebooks

In [ ]:
%store

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();